In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

In [ ]:
def prepare_classic_dataset():
    # load dataset
    x, y = np.load('data_mfpt_3_classes.npz')['x'], np.load('data_mfpt_3_classes.npz')['y']
    # separate into test and train
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
    # normalize
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    # one-hot encode
    y_train = np.eye(3)[y_train]
    y_test = np.eye(3)[y_test]
    
    return x_train, x_test, y_train, y_test
    
x_train, x_test, y_train, y_test = prepare_classic_dataset()


def prepare_quantum_dataset(x):
    """angle encoding"""
    # Qubits
    q = cirq.GridQubit.rect(1, len(x))
    # Operations
    ops = [cirq.ry(2 * x[i]).on(q[i]) for i in range(len(x))]
    # 1 circuit == 1 datapoint
    circuit = cirq.Circuit(ops)
    
    
    
    return circuit

x_train_q = [prepare_quantum_dataset(x) for x in x_train]
x_test_q = [prepare_quantum_dataset(x) for x in x_test]

SVGCircuit(x_train_q[1])

In [ ]:
def train():
    # load & prepare dataset
    x_train, x_test, y_train, y_test = prepare_classic_dataset()
    x_train_q = [prepare_quantum_dataset(x) for x in x_train]
    x_test_q = [prepare_quantum_dataset(x) for x in x_test]


    # generate model
    
    # Parameters
    params = sympy.symbols('a b c')
    # Qubits
    q = cirq.GridQubit.rect(1, 5)
    # Operations
    ops = []
    for i in range(5):
        ops.append(cirq.ry(params[0]).on(q[i]))
        ops.append(cirq.rx(params[1]).on(q[i]))
        ops.append(cirq.rz(params[2]).on(q[i]))
    # PQC circuit model
    model_circuit = cirq.Circuit(ops)
    
    # The classical neural network layers.
    nn = tf.keras.Sequential([tf.keras.layers.Dense(10, activation='elu'),
                              tf.keras.layers.Dense(3, activation='softmax')])
    # Circuit's input
    circuit_input = tf.keras.Input(shape=(), dtype=tf.string, name='circuits_input')

    # TFQ layer for circuits.
    measurement_ops = [cirq.Z(q[i]) for i in range(5)]
    circuit_layer = tfq.layers.PQC(model_circuit, measurement_ops)

    # The Keras model
    model = tf.keras.Model(inputs=circuit_input, outputs=nn(circuit_layer(circuit_input)))
    
    # Training
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x=tfq.convert_to_tensor(x_train_q),
                        y=y_train,
                        epochs=50,
                        verbose=1)
    
    # Evaluate
    results_train = model.evaluate(tfq.convert_to_tensor(x_train_q), y_train)
    results_test = model.evaluate(tfq.convert_to_tensor(x_test_q), y_test)
    train_loss, train_accuracy = results_train[0], results_train[1]
    test_loss, test_accuracy = results_test[0], results_test[1]
    
    return {'train_accuracy': results_train[1],
           'test_accuracy': results_test[1],
           'train_loss': results_train[0],
           'test_loss': results_test[0]}

In [ ]:
# Parameters
params = sympy.symbols('a b c')
# Qubits
q = cirq.GridQubit.rect(1, 5)
# Operations
ops = []
for i in range(5):
    ops.append(cirq.ry(params[0]).on(q[i]))
    ops.append(cirq.rx(params[1]).on(q[i]))
    ops.append(cirq.rz(params[2]).on(q[i]))
# PQC circuit model
model_circuit = cirq.Circuit(ops)

SVGCircuit(model_circuit)

In [ ]:
# The classical neural network layers.
nn = tf.keras.Sequential([tf.keras.layers.Dense(10, activation='elu'),
                          tf.keras.layers.Dense(3, activation='softmax')])
# Circuit's input
circuit_input = tf.keras.Input(shape=(), dtype=tf.string, name='circuits_input')

# TFQ layer for circuits.
measurement_ops = [cirq.Z(q[i]) for i in range(5)]
circuit_layer = tfq.layers.PQC(model_circuit, measurement_ops)

# The Keras model
model = tf.keras.Model(inputs=circuit_input, outputs=nn(circuit_layer(circuit_input)))

In [ ]:
# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=tfq.convert_to_tensor(x_train_q),
                    y=y_train,
                    epochs=150,
                    verbose=1)

In [ ]:
model.evaluate(tfq.convert_to_tensor(x_test_q), y_test)
y_test_ne = np.argmax(y_test, axis=1)
y_test_pred_ne = np.argmax(model.predict(tfq.convert_to_tensor(x_test_q)), axis=1)
cm = confusion_matrix(y_test_ne, y_test_pred_ne)
print(cm)

In [ ]:
from utils import plot_confusion_matrix
plot_confusion_matrix(cm, ['ND', 'OR', 'IR'], title='MFPT Classification Case Study: Test Data', normalize=True)

In [ ]:
plt.plot(history.history['loss'])
plt.title("MFPT Classification Case Study")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.title("MFPT Classification Case Study")
plt.xlabel("Epochs")
plt.ylabel("Accuracy [%]")
plt.grid()
plt.show()

In [ ]:
# Parameters
params = sympy.symbols('x1 x2 x3 x4 x5')
# Qubits
q = cirq.GridQubit.rect(1, 5)
# Operations
ops = []
for i in range(5):
    ops.append(cirq.ry(params[i]).on(q[i]))
# PQC circuit model
model_circuit = cirq.Circuit(ops)

SVGCircuit(model_circuit)